In [6]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sys
sys.tracebacklimit = 0

import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt

In [8]:
from pettingzoo.test import parallel_api_test
from solution.custom_gym import CustomGymEnviornment
from solution.trainer import *
from solution.policy_net import *

In [9]:
from core.agent import *
from core.world import * 
from dynamics.space.resource_map import *
from core.render import * 
from core.env_params import * 
from core.skill import * 
from core.models import *
from core.env_params import *
from core.message import *

In [10]:
from sar.sar_agent import *
from dynamics.agents.trade_agent import * 
from sar.sar_world import *
from dynamics.models.util_sampler import *

def initialize_swarm(world : BaseWorld):
    swarm = [SARAgent() for i in range(SWARM_SIZE)]
    for agent in swarm:
        agent.set_utility(SARUtilityFunction())
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)


In [11]:

from sar.urban_gen import * 
from sar.victims import * 
from sar.sar_comm import * 

terrain_generator = UrbanTerrainMapGenerator(padding = MAX_VISIBILITY)
victim_generator = VictimGenerator(padding = MAX_VISIBILITY)
def initialize_terrain(world : BaseWorld):
    terrain_map, population_map = terrain_generator.generate(world._dims)
    map_collection : BaseMapCollection = BaseMapCollection()
    map_collection.add_map("Terrain", terrain_map)
    map_collection.add_map("Population", population_map)

    victim_generator.set_density_map(population_map)
    victim_map = victim_generator.generate(world._dims)

    map_collection.add_map("Victims", victim_map)
    return map_collection


In [12]:
from sar.energy import EnergyModel
from sar.victims import VictimModel
from solution.sar_action_interpreter import *

world = SARWorld(dims = WORLD_DIMS,
              swarm_initializer= initialize_swarm,
              generation_pipeline=initialize_terrain
              )
world.add_model("energy_model", EnergyModel())
world.add_model("victim_model", VictimModel())
world.reset()

comms_protocol = SARCommunicationProtocol()
action_interpreter = SARActionInterpreter()

In [13]:
custom_gym : CustomGymEnviornment = CustomGymEnviornment(world, action_interpreter)

In [14]:
custom_gym.reset(42)

({1: {'Victims': array([[1., 0., 1., 0., 1., 1., 1.],
          [0., 0., 1., 0., 0., 1., 1.],
          [0., 0., 1., 1., 0., 1., 0.],
          [0., 1., 1., 0., 1., 0., 1.],
          [1., 1., 1., 1., 1., 1., 1.],
          [1., 0., 1., 1., 1., 1., 1.],
          [1., 0., 1., 0., 1., 1., 1.]])},
  2: {'Victims': array([[0., 1., 1., 1., 0., 1., 1.],
          [0., 0., 0., 1., 1., 1., 0.],
          [1., 0., 1., 0., 0., 1., 0.],
          [0., 1., 0., 0., 0., 1., 1.],
          [1., 0., 1., 0., 0., 1., 0.],
          [0., 0., 1., 1., 0., 1., 1.],
          [0., 0., 0., 1., 0., 1., 0.]])},
  3: {'Victims': array([[1., 1., 1., 1., 1., 0., 1.],
          [0., 1., 1., 0., 1., 1., 1.],
          [1., 0., 0., 1., 1., 1., 1.],
          [1., 1., 0., 0., 0., 0., 0.],
          [0., 1., 1., 1., 0., 0., 1.],
          [1., 1., 0., 0., 1., 0., 0.],
          [1., 0., 1., 1., 0., 1., 1.]])},
  4: {'Victims': array([[0., 0., 1., 0., 1., 0., 1.],
          [0., 0., 1., 1., 1., 1., 0.],
          [1., 

### Testing

In [15]:
parallel_api_test(custom_gym, num_cycles=1_000_000)
custom_gym.reset()

Passed Parallel API test


({1: {'Victims': array([[1., 0., 1., 0., 1., 1., 1.],
          [0., 0., 1., 0., 0., 1., 1.],
          [0., 0., 1., 1., 0., 1., 0.],
          [0., 1., 1., 0., 1., 0., 1.],
          [1., 1., 1., 1., 1., 1., 1.],
          [1., 0., 1., 1., 1., 1., 1.],
          [1., 0., 1., 0., 1., 1., 1.]])},
  2: {'Victims': array([[0., 1., 1., 1., 0., 1., 1.],
          [0., 0., 0., 1., 1., 1., 0.],
          [1., 0., 1., 0., 0., 1., 0.],
          [0., 1., 0., 0., 0., 1., 1.],
          [1., 0., 1., 0., 0., 1., 0.],
          [0., 0., 1., 1., 0., 1., 1.],
          [0., 0., 0., 1., 0., 1., 0.]])},
  3: {'Victims': array([[1., 1., 1., 1., 1., 0., 1.],
          [0., 1., 1., 0., 1., 1., 1.],
          [1., 0., 0., 1., 1., 1., 1.],
          [1., 1., 0., 0., 0., 0., 0.],
          [0., 1., 1., 1., 0., 0., 1.],
          [1., 1., 0., 0., 1., 0., 0.],
          [1., 0., 1., 1., 0., 1., 1.]])},
  4: {'Victims': array([[0., 0., 1., 0., 1., 0., 1.],
          [0., 0., 1., 1., 1., 1., 0.],
          [1., 

# Training

In [16]:

from models.base import * 
from models.idqn import * 
from solution.policy_net import PolicyNet

In [17]:
model = IDQN(env = custom_gym,
             feature_extractor= feature_extractor,
             policy_net= PolicyNet(1, 7, 12), 
             target_net=  PolicyNet(1, 7, 12))

In [18]:
train_loop(custom_gym, model, games=10, seed=42)

Training on thesis.
{1: {'Victims': array([[1., 0., 1., 0., 1., 1., 1.],
       [0., 0., 1., 0., 0., 1., 1.],
       [0., 0., 1., 1., 0., 1., 0.],
       [0., 1., 1., 0., 1., 0., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1.],
       [1., 0., 1., 0., 1., 1., 1.]])}, 2: {'Victims': array([[0., 1., 1., 1., 0., 1., 1.],
       [0., 0., 0., 1., 1., 1., 0.],
       [1., 0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 1.],
       [1., 0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 1., 0., 1., 1.],
       [0., 0., 0., 1., 0., 1., 0.]])}, 3: {'Victims': array([[1., 1., 1., 1., 1., 0., 1.],
       [0., 1., 1., 0., 1., 1., 1.],
       [1., 0., 0., 1., 1., 1., 1.],
       [1., 1., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 1.],
       [1., 1., 0., 0., 1., 0., 0.],
       [1., 0., 1., 1., 0., 1., 1.]])}, 4: {'Victims': array([[0., 0., 1., 0., 1., 0., 1.],
       [0., 0., 1., 1., 1., 1., 0.],
       [1., 0., 1., 1., 1., 1., 1.],
       [0., 1., 1., 0.

KeyboardInterrupt: 